In [ ]:
pip install kagglehub pandas matplotlib seaborn scikit-learn

In [ ]:
import pandas as pd
import glob
import os


RAW_DATA_PATH = '../data/raw/'
all_csv_files = glob.glob(os.path.join(RAW_DATA_PATH, "*.csv"))
list_of_dfs = []

for file_path in all_csv_files:
    print(f"Wczytywanie: {os.path.basename(file_path)}...")
    try:
        df_temp = pd.read_csv(
            file_path,
            encoding='latin1',
            low_memory=False
        )
        list_of_dfs.append(df_temp)
    except Exception as e:
        print(f"Błąd przy wczytywaniu {file_path}: {e}")

if not list_of_dfs:
    print("Żaden plik nie został wczytany.")
else:
    df_combined = pd.concat(list_of_dfs, axis=0, ignore_index=True)

    print("\n--- Pliki połączone ---")

    if ' Label' in df_combined.columns:
        print("\nRozkład etykiet w połączonym zbiorze:")
        print(df_combined[' Label'].value_counts())
    else:
        print("\nUWAGA: Nie znaleziono kolumny ' Label'. Sprawdź nazwy kolumn.")

    print("\nPierwsze 5 wierszy połączonych danych:")
    display(df_combined.head())

In [ ]:
print(f"Oryginalny kształt danych (wiersze, kolumny): {df_combined.shape}")
original_rows = df_combined.shape[0]
print("\nTypy danych w kolumnach (fragment): ---")
print(df_combined.info(verbose=False))

label_column = ' Label'

if label_column in df_combined.columns:
    print(f"\nRozkład klas (ataków i ruchu normalnego)")
    print(df_combined[label_column].value_counts())
else:
    print(f"\nBŁĄD: Nie znaleziono kolumny '{label_column}'")

print("\nStatystyki opisowe (próbka):")
display(df_combined.describe())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = df_combined.copy()

original_rows = df.shape[0]
original_cols = df.shape[1]
print(f"Początkowy rozmiar danych: {df.shape}")

label_column = ' Label'
target_labels = ['BENIGN', 'DoS Hulk', 'FTP-Patator']

print(f"\nFiltrowanie do etykiet: {target_labels}...")
df = df[df[label_column].isin(target_labels)]
print(f"Rozmiar po filtrowaniu etykiet: {df.shape}")

print("\nRozpoczynanie czyszczenia danych...")

df.replace([np.inf, -np.inf], np.nan, inplace=True)
rows_before_na = df.shape[0]
df.dropna(inplace=True)
rows_after_na = df.shape[0]
print(f"Usunięto {rows_before_na - rows_after_na} wierszy z wartościami NaN lub Inf.")

rows_before_neg = df.shape[0]
df = df[df[' Flow Duration'] >= 0]
rows_after_neg = df.shape[0]
print(f"Usunięto {rows_before_neg - rows_after_neg} wierszy z ujemnym czasem trwania.")

rows_before_dup = df.shape[0]
df.drop_duplicates(inplace=True)
rows_after_dup = df.shape[0]
print(f"Usunięto {rows_before_dup - rows_after_dup} zduplikowanych wierszy.")
final_rows_after_cleaning = df.shape[0]

df['Binary_Label'] = df[label_column].apply(lambda x: 0 if x == 'BENIGN' else 1)
df = df.drop(columns=[label_column])

print("\nRozpoczynanie redukcji wymiarowości...")

metadata_cols = ['Flow ID', ' Source IP', ' Destination IP', ' Timestamp', ' Destination Port']
existing_metadata_cols = [col for col in metadata_cols if col in df.columns]
df = df.drop(columns=existing_metadata_cols)
print(f"Usunięto {len(existing_metadata_cols)} kolumn z metadanymi.")
y = df['Binary_Label']
X = df.drop(columns=['Binary_Label'])
cols_before_const = X.shape[1]
constant_cols = X.columns[X.nunique() == 1]
X = X.drop(columns=constant_cols)
cols_after_const = X.shape[1]
print(f"Usunięto {len(constant_cols)} stałych kolumn (np. 'Bwd PSH Flags').")
print(f"Lista usuniętych stałych kolumn: {list(constant_cols)}")

final_cols_after_reduction = X.shape[1]

print("\nDzielenie na zbiory treningowe i testowe...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Zbiór treningowy: {X_train.shape[0]} próbek")
print(f"Zbiór testowy:    {X_test.shape[0]} próbek")
print("\nNormalizowanie danych (MinMaxScaler)...")
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

print("Pre-processing zakończony")

In [ ]:
print("\n\nWYNIK: Próbka (pierwsze 5 wierszy) przetworzonych danych treningowych")
display(X_train_scaled.head())

print("\n\nWYNIK: Statystyki przetworzonych danych (potwierdzenie normalizacji)")
display(X_train_scaled.describe())

PROCESSED_DATA_PATH = '../data/processed'
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)
print(f"Pliki zostaną zapisane w folderze: '{PROCESSED_DATA_PATH}/'")

X_train_scaled.to_csv(os.path.join(PROCESSED_DATA_PATH, "X_train_scaled.csv"), index=False)
y_train.to_csv(os.path.join(PROCESSED_DATA_PATH, "y_train.csv"), index=False)

X_test_scaled.to_csv(os.path.join(PROCESSED_DATA_PATH, "X_test_scaled.csv"), index=False)
y_test.to_csv(os.path.join(PROCESSED_DATA_PATH, "y_test.csv"), index=False)

print("Przygotowywanie danych dla autoenkodera (tylko ruch normalny)...")

train_data_scaled = X_train_scaled.copy()
train_data_scaled['Binary_Label'] = y_train.values

autoencoder_train_data = train_data_scaled[train_data_scaled['Binary_Label'] == 0]
autoencoder_train_data = autoencoder_train_data.drop(columns=['Binary_Label'])

autoencoder_train_data.to_csv(os.path.join(PROCESSED_DATA_PATH, "autoencoder_train_data.csv"), index=False)

print(f"Zapisano 'X_train_scaled.csv' i 'y_train.csv' w '{PROCESSED_DATA_PATH}'.")
print(f"Zapisano 'X_test_scaled.csv' i 'y_test.csv' w '{PROCESSED_DATA_PATH}'.")
print(f"Zapisano 'autoencoder_train_data.csv' w '{PROCESSED_DATA_PATH}'.")